#### Calculate Embeddings of All Photos in the Database

In [ ]:
data_transforms = transforms.Compose([
transforms.Resize(size=(160,160)),
transforms.ToTensor(),
transforms.Normalize(
    mean=[0.6025, 0.4704, 0.4078], 
    std=[0.2493, 0.2278, 0.2257]
)
])

model.eval()
def get_embedding(path):
  img = Image.open(path)
  img = data_transforms(img)
  img = img.unsqueeze(0)
  img = img.cuda()
  emb = model(img).cpu().squeeze().detach()
  del img
  torch.cuda.empty_cache()
  return emb

embeddings_complemented = {}
MAX_SESSIONS_ADDED = 1
for enroll_id in set(sessions_dataset['identity']):
  embeddings_complemented[enroll_id] = []
  full_path = os.path.join(ENROLL_IMGS_PATH, f'E{enroll_id:04d}.png')
  embeddings_complemented[enroll_id].append(get_embedding(full_path))
  for day in range(0, MAX_SESSIONS_ADDED):
    for instance in sessions_dataset['id']:
      instance_day = sessions_dataset['day'][instance]
      instance_identity = sessions_dataset['identity'][instance]
      if instance_day != day or instance_identity != enroll_id:
        continue
      photo_path = f"{QUERY_IMGS_PATH}/Q{instance:04d}.png"
      embeddings_complemented[enroll_id].append(get_embedding(photo_path))
